In [96]:
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install scipy

import os

SIRPath = "/home/het/Documents/github-projects/SIR-MODEL"
print(os.getcwd())
os.chdir(SIRPath)
print(os.getcwd())

import requests 
from sys import platform
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import json
import math
from sys import platform
from scipy.optimize import curve_fit

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
/home/het/Documents/github-projects/SIR-MODEL
/home/het/Documents/github-projects/SIR-MODEL


In [97]:
## IsWindows defintion

def isWindows():
    return platform.lower().startswith("win")

In [98]:
## urlwrite function.
print(os.getcwd())
def urlwrite(url, fileName):
    response = requests.get(url)

    for i in range(5):
        if response.status_code == 200:
            break
        response = requests.get(url)
    else:
        raise Exception("Internet not working")
    
    with open(fileName, 'w') as fptr:
        fptr.write(response.text)


/home/het/Documents/github-projects/SIR-MODEL


In [99]:
## Required constants

dummyFileName = 'data\dummy.csv' if isWindows() else 'data/dummy.csv'
DateFormat = '%Y-%m-%d'
fileName = 'data\cityAndStateNames.json' if isWindows() else 'data/cityAndStateNames.json'

In [100]:
## Get district names and get state names
def getDistrictNames():
    with open(fileName) as fptr:
        return json.loads(fptr.read())['cityNames']

def getStateNames():
    with open(fileName) as fptr:
        return json.loads(fptr.read())['stateNames']

In [101]:
## Movmean definition 

def movmean(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [102]:
## Init class for init element

class Init:
    def __init__(self, country, date, C, D, R, Npop):
        self.country = country
        self.date = date
        self.C = C
        self.D = D
        self.R = R
        self.Npop = Npop

In [103]:
## Model class for EXP and LOG elements

class ModelClass:
    def __init__(self, date, C):
        self.date = date
        self.C = C

In [104]:
ifplotPR = True
dists = True
debug = False
ifprint = True
span = 7

In [105]:
fullName, sb_number,  name = None, None, None 

In [106]:
if dists:
    fullName = 'https://api.covid19india.org/csv/latest/districts.csv'
    name = getDistrictNames()
    tt1 = pd.to_datetime('2020-4-26')
    sb_number = 4
else:
    fullName = 'https://api.covid19india.org/csv/latest/states.csv'
    name = getStateNames()
    tt1 = pd.to_datetime('2020-3-14')
    sb_number = 5

In [107]:
urlwrite(fullName, dummyFileName)

tableAll = pd.read_csv(dummyFileName, na_values={'Tested':['']})
tableAll.Date = pd.to_datetime(tableAll.Date, format="%Y-%m-%d")

span1 = 14

mean_si, min_mean_si, max_mean_si = 4.7, 3.7, 6.0
std_si, min_std_si, max_std_si = 2.9, 1.9, 4.9 

In [108]:
## Exponential model function.

def funcExp1(x, a, b):
    return a*np.exp(b*x)

In [109]:
## Logistic model function.

def funcLogistic(x, a1, a2, a3):
    return a1/(1 + a2*np.exp(-1*a3*x))

In [110]:
## logmdl model function.

def funcLogmdl(x, a0, a1, a2):
    return a0/(1+a1*np.exp(-1*a2*x))

In [111]:
for n in range(len(name)):

    Location = name[n]

    # To avoid scope related problems.
    tableLocation = None
    R, D, C, O, T = None, None, None, None, None

    dateInit = pd.to_datetime('2020-03-01', format = DateFormat)

    if dists:
        tableLocation = tableAll[(tableAll.District == Location) & (tableAll.Date >= pd.to_datetime('1-3-2020', format='%d-%m-%Y'))]

        R = tableLocation['Recovered'].to_numpy()
        D = tableLocation['Deceased'].to_numpy()
        C = tableLocation['Confirmed'].to_numpy()
        O = tableLocation['Other'].to_numpy()
        T = tableLocation['Tested'].to_numpy()
    else:
        tableLocation = tableAll[(tableAll.State == Location) & (tableAll.Date >= pd.to_datetime('1-3-2020', format='%d-%m-%Y'))]

        R = tableLocation['Recovered'].to_numpy()
        D = tableLocation['Deceased'].to_numpy()
        C = tableLocation['Confirmed'].to_numpy()
        O = tableLocation['Other'].to_numpy()
        T = tableLocation['Tested'].to_numpy()
 
    date = tableLocation['Date'].to_numpy()
    A = C - D - O - R # Active cases

    dailyC = np.diff(C)

    if debug:
        fig = plt.figure()
        fig.add_subplot(2, 1, (1))

        plt.stem(date, C)
        plt.xlabel('Date')
        plt.ylabel('Confirmed Cases')
            
        fig.add_subplot(2, 1, (2))

        plt.stem(date[1:], dailyC)
        plt.xlabel('Date')
        plt.ylabel('Infection rate')

        plt.show()
    
    if not dists:
        PR = (np.diff(movmean(C,span)) / np.diff(movmean(T,span))) *100
        Testing = np.diff(T)
    CFR = (np.diff(movmean(D,span)) / np.diff(movmean(C,span))) *100

    date0 = np.copy(date)
    tt1 = None

    if dists:
        tt1 = str(date0[0])
        sb_number = 4
    else:
        tt1 = str(date0[0])
        sb_number = 5
        
    date1 = pd.to_datetime('2021-4-1', format = DateFormat)

    dateEnd = date[-1]

    tableLocation1 = tableLocation[(tableLocation.Date > date1) & (tableLocation.Date < dateEnd)]

    date = tableLocation1['Date'].to_numpy()
    C = tableLocation1['Confirmed'].to_numpy()
    D = tableLocation1['Deceased'].to_numpy()
    R = tableLocation1['Recovered'].to_numpy()
    Npop = int('80e6', 16)

    init = Init(Location, date, C, D, R, Npop)
    # Init is a simple class to save data like a struct object.

    diffC = np.diff(init.C)

    q1 = diffC[0]
    q2 = diffC[-1]
    t = diffC.shape[0]          

In [112]:
    ## Exponential model

    Cexp = np.copy(init.C)
    texp = np.array(list(range(1, Cexp.shape[0] + 1)))

    f, pcov = curve_fit(funcExp1, texp, Cexp)

    coeff = list(f)
    td_2 = math.log(2)/coeff[1]

    tnew = texp + 120

    EXP = ModelClass(date, funcExp1(tnew, *f))


In [113]:
    ## Logistic model

    flog, pcov = curve_fit(funcLogistic, texp, Cexp)

In [114]:
    ## Using For coefficients
        
    logmdl, pcov= curve_fit(funcLogmdl, texp, Cexp)
    LOG = ModelClass(date, funcLogmdl(tnew, *logmdl))
